Import Libraries

In [1]:

import pandas as pd
import numpy as np 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder 

Import keyword data

In [2]:
keyword_df = pd.read_csv('PM_Keywords_granular.csv')


In [3]:
keyword_df.head()

,Unnamed: 0,keyword,Score,POS,job_ID,Company,Title,query,time
0,0,challenges,0.105,Noun,1,Family Service League,IT Project Manager,IT Project Manager,2020-02-10 21:09:49.586156
1,1,standards,0.101,Noun,1,Family Service League,IT Project Manager,IT Project Manager,2020-02-10 21:09:49.586156
2,2,determine,0.096,Noun,1,Family Service League,IT Project Manager,IT Project Manager,2020-02-10 21:09:49.586156
3,3,tasks,0.095,Noun,1,Family Service League,IT Project Manager,IT Project Manager,2020-02-10 21:09:49.586156
4,4,coordinate,0.095,Noun,1,Family Service League,IT Project Manager,IT Project Manager,2020-02-10 21:09:49.586156


Import csv with count of keywords

In [4]:
keyword_summary_df = pd.read_csv('PM_Keywords_summary.csv')
keyword_summary_df.head()

,Unnamed: 0,keyword,POS,time,query,keyword_count
0,148,internal,Adjective,2020-02-10 21:09:49.586156,IT Project Manager,10
1,353,technical,Adjective,2020-02-10 21:09:49.586156,IT Project Manager,9
2,294,schedules,Noun,2020-02-10 21:09:49.586156,IT Project Manager,6
3,58,create,Noun,2020-02-10 21:09:49.586156,IT Project Manager,6
4,325,standards,Noun,2020-02-10 21:09:49.586156,IT Project Manager,6


Import Job Salaries (manually looked up on indeed)

In [5]:
salary_df = pd.read_csv('job_avg_salaries.csv')

In [6]:
salary_df

,Job_Tittle,Domain,Date,Avg_Salary
0,Technology Project Manager,Information Technology,2020-02-09,103175
1,Technology Product Manager,Information Technology,2020-02-09,106838
2,Data Scientist,Information Technology,2020-02-09,123253
3,DevOps Engineer,Information Technology,2020-02-09,126982
4,Software Engineer,Information Technology,2020-02-09,107254
5,Data Engineer,Information Technology,2020-02-09,127179
6,Solutions Architect,Information Technology,2020-02-09,139443
7,Data Analyst,Information Technology,2020-02-09,71274
8,Full Stack Developer,Information Technology,2020-02-09,113481
9,Development Manager,Information Technology,2020-02-09,143371


In [7]:
top_50_keywords = keyword_summary_df["keyword"][:50].tolist()
top_50_keywords

['internal',
 'technical',
 'schedules',
 'create',
 'standards',
 'shared internal',
 'updates consistent',
 'partners subject',
 'matter authorities',
 'coordinates',
 'parties vendors',
 'coordinate',
 'manager operations',
 'possess',
 'vendors flawless',
 'seeing projects',
 'manager needed',
 'determine',
 'key partners',
 'implementation management',
 'needed',
 'timelines',
 'expected',
 'executive management',
 'stakeholders strategy',
 'timeline hire',
 'track costs',
 'erp manager',
 'future processes',
 'erp',
 'needed manage',
 'netsuite',
 'strong familiarity',
 'software tools',
 'functionality',
 'type renewal',
 'strategy adjustments',
 'resources parties',
 'procedure',
 'reasonable accommodation',
 'internal resources',
 'accommodation',
 'progress',
 'report escalate',
 'assistance manager',
 'challenges',
 'tasks',
 'knowledgeable microsoft',
 'adjustments',
 'issues management']

Filter only the top keywords

In [8]:

keyword_df= keyword_df[keyword_df['keyword'].isin(top_50_keywords)]

Calculate job ID Score for the top 50 keywords for a given job 

In [29]:
job_id_scores_df = pd.DataFrame()
job_id_scores_df= keyword_df[['job_ID','Company', 'Score']]
job_id_scores_df = job_id_scores_df.groupby(['job_ID','Company'], as_index=False)['Score'].agg({"sum": "sum"}).sort_values(['sum'], ascending=False)
job_id_scores_df.head()

,job_ID,Company,sum
32,87,Amplify Education,1.193
31,73,Amplify Education,1.193
6,19,Amplify Education,1.193
7,20,Amplify Education,1.193
24,56,Amplify Education,1.193


Build out sparse matrix with keywords as a columns

In [30]:
df2 = pd.get_dummies(df_with_keyword_catagories['keyword']).to_sparse(0)
df2['job_ID'] = keyword_df['job_ID']
df2['Company'] = keyword_df['Company']
df2['Title'] = keyword_df['Title']
df2['Score'] = keyword_df['Score']
df2.head()

C:\Users\doug\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.to_sparse is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
C:\Users\doug\Anaconda3\lib\site-packages\pandas\core\frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)
C:\Users\doug\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:745: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  default_fill_value=self._default_fill_value,
C:\Users\doug\Anaconda3\li

,accommodation,adjustments,assistance manager,challenges,coordinate,coordinates,create,determine,erp,erp manager,...,timeline hire,timelines,track costs,type renewal,updates consistent,vendors flawless,job_ID,Company,Title,Score
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,Family Service League,IT Project Manager,0.105
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,Family Service League,IT Project Manager,0.101
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,Family Service League,IT Project Manager,0.096
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,Family Service League,IT Project Manager,0.095
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,Family Service League,IT Project Manager,0.095


In [33]:
feature_matrix = df2.groupby(['job_ID','Company','Title']).sum()
feature_matrix.head()

C:\Users\doug\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:745: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  default_fill_value=self._default_fill_value,
C:\Users\doug\Anaconda3\lib\site-packages\pandas\core\generic.py:3606: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  result = self._constructor(new_data).__finalize__(self)
C:\Users\doug\Anaconda3\lib\site-packages\pandas\core\frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.py

,,,accommodation,adjustments,assistance manager,challenges,coordinate,coordinates,create,determine,erp,erp manager,...,strong familiarity,tasks,technical,timeline hire,timelines,track costs,type renewal,updates consistent,vendors flawless,Score
job_ID,Company,Title,,,,,,,,,,,,,,,,,,,,,
1,Family Service League,IT Project Manager,0,0,0,1,1,0,1,1,0,0,...,0,1,1,0,1,0,0,0,0,0.917
5,Combined Computer Resources,Project Manager with a Big data background,0,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,0,1.053
7,Leukemia & Lymphoma Society,IT Project Manager,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.819
16,Memorial Sloan-Kettering Cancer Center,Project Manager - Translational Informatics,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.167
17,Oneida Nation Enterprises,IT Project Manager - Junior or Senior,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.817
